# __Daily Social Network Analysis__

In [1]:
import datetime
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib widget

In [2]:
h_df = pd.read_parquet('data/tweets/week_05/day_7_2021-05-29_hashtags_ext.parquet')
h_df.shape

(131549, 27)

In [3]:
G = nx.MultiDiGraph()
nodes = []
# adding nodes
interaction_df = h_df.query("inReplyToUser != 'None' or quotedTweet != 'None'")[['user', 'inReplyToUser', 'quotedTweet']]

#for user in h_df['user']:
for user in interaction_df['user']:
    user = eval(user)
    node = (user['username'], user['description'])
    if node not in nodes:
        nodes.append(node)
        
print(f'{len(nodes)} nodes...')

21983 nodes...


In [4]:
for node in nodes:
    G.add_node(node[0], description=node[1])
    
list(G.nodes(data=True))[:5]

[('eliserohrig', {'description': ''}),
 ('AssisMartinsBM', {'description': 'Bolsonaro 2022 - Presidente do Brasil'}),
 ('Sergio29562298',
  {'description': 'Sempre uma nova descoberta. A cada dia um novo aprendizado.'}),
 ('ErikakHilton',
  {'description': 'Mulher negra, trans, militante.\nPresidenta da Comissão de Direitos Humanos da Câmara de SP.\n\nA Vereadora + votada do Brasil em 2020. #VidasNegrasImportam #PSOL'}),
 ('B51Syrena',
  {'description': 'Sou Bolsonaro da cabeça aos pés. Não respondo DM 🇧🇼🇧🇼🇧🇼🇧🇼🇧🇼🇧🇼🇧🇼🇧🇼🇧🇼'})]

In [5]:
interaction_df = h_df.query("inReplyToUser != 'None' or quotedTweet != 'None'")[['user', 'inReplyToUser', 'quotedTweet']]
interaction_df['user'] = interaction_df['user'].map(lambda x: eval(x))
interaction_df['inReplyToUser'] = interaction_df['inReplyToUser'].map(lambda x: eval(x))
interaction_df['quotedTweet'] = interaction_df['quotedTweet'].map(lambda x: eval(x))

interaction_df.shape

(63736, 3)

In [6]:
edges = []

for i in range(interaction_df.shape[0]):
    intr = interaction_df.iloc[i]
    if intr['inReplyToUser']:
        edges.append((intr['user']['username'], intr['inReplyToUser']['username'], 'reply'))
    if intr['quotedTweet']:
        edges.append((intr['user']['username'], intr['quotedTweet']['user']['username'], 'retweet'))

len(edges)

63942

In [7]:
edges_count = pd.Series(edges).value_counts()
edges_count

(luizcarlosprsp, MariaCleideque1, reply)    100
(brasildefato, brasildefato, reply)          58
(Susan_Blum, Susan_Blum, retweet)            55
(ANTIBOLSONARO16, OABurgos, reply)           35
(vasilva1971, MaryamSCI, reply)              32
                                           ... 
(vvecchi, nadanovonofront, reply)             1
(kezya_95, BuzzFeedBrasil, reply)             1
(Malutrin2018, rvianna, reply)                1
(Lisete97371827, solac1991, reply)            1
(el_cardozito, nisisatanas, reply)            1
Length: 57243, dtype: int64

In [8]:
weighted_edges = []

for i in range(edges_count.shape[0]):
    weighted_edges.append(edges_count.index[i]+(edges_count.values[i],))
    
weighted_edges[:5]

[('luizcarlosprsp', 'MariaCleideque1', 'reply', 100),
 ('brasildefato', 'brasildefato', 'reply', 58),
 ('Susan_Blum', 'Susan_Blum', 'retweet', 55),
 ('ANTIBOLSONARO16', 'OABurgos', 'reply', 35),
 ('vasilva1971', 'MaryamSCI', 'reply', 32)]

In [9]:
# adding edges
for edge in weighted_edges:
    G.add_edge(edge[0], edge[1], weight=edge[3], int_type=edge[2])
    
list(G.edges(data=True))[:10]

[('eliserohrig', 'ccandradeRJ', {'weight': 2, 'int_type': 'reply'}),
 ('eliserohrig', 'UOLNoticias', {'weight': 1, 'int_type': 'retweet'}),
 ('eliserohrig', 'wasdanna', {'weight': 1, 'int_type': 'reply'}),
 ('AssisMartinsBM', 'AndreRayol1', {'weight': 1, 'int_type': 'reply'}),
 ('AssisMartinsBM', 'biiacst_', {'weight': 1, 'int_type': 'reply'}),
 ('AssisMartinsBM', 'g1', {'weight': 1, 'int_type': 'reply'}),
 ('AssisMartinsBM', 'mfriasoficial', {'weight': 1, 'int_type': 'reply'}),
 ('AssisMartinsBM', 'EspiritoQue', {'weight': 1, 'int_type': 'reply'}),
 ('Sergio29562298', 'GuilhermeBoulos', {'weight': 1, 'int_type': 'reply'}),
 ('ErikakHilton', 'ErikakHilton', {'weight': 1, 'int_type': 'reply'})]

In [10]:
G_reply = nx.MultiDiGraph((source, target, attr) for source, target, attr in G.edges(data=True) if attr['int_type'] == 'reply')
G_rt = nx.MultiDiGraph((source, target, attr) for source, target, attr in G.edges(data=True) if attr['int_type'] == 'retweet')

list(G_reply.edges(data=True))[:10]

[('eliserohrig', 'ccandradeRJ', {'weight': 2, 'int_type': 'reply'}),
 ('eliserohrig', 'wasdanna', {'weight': 1, 'int_type': 'reply'}),
 ('ccandradeRJ', 'eliserohrig', {'weight': 2, 'int_type': 'reply'}),
 ('ccandradeRJ', 'isamelissams', {'weight': 1, 'int_type': 'reply'}),
 ('ccandradeRJ', 'liberdade_ldr', {'weight': 1, 'int_type': 'reply'}),
 ('ccandradeRJ', 'CarlosS48442375', {'weight': 1, 'int_type': 'reply'}),
 ('ccandradeRJ', 'MaClara14373139', {'weight': 1, 'int_type': 'reply'}),
 ('ccandradeRJ', 'denise47613384', {'weight': 1, 'int_type': 'reply'}),
 ('ccandradeRJ', 'MarcosQuezado1', {'weight': 1, 'int_type': 'reply'}),
 ('ccandradeRJ', 'Rconstantino', {'weight': 1, 'int_type': 'reply'})]

In [12]:
len(G_reply)

19547

In [13]:
list(G_rt.edges(data=True))[:10]

[('eliserohrig', 'UOLNoticias', {'weight': 1, 'int_type': 'retweet'}),
 ('ErikakHilton', 'kerlinecardoso', {'weight': 1, 'int_type': 'retweet'}),
 ('B51Syrena', 'fcspjr81', {'weight': 1, 'int_type': 'retweet'}),
 ('B51Syrena', 'profcabarros', {'weight': 1, 'int_type': 'retweet'}),
 ('B51Syrena', 'OAntagoniza11', {'weight': 1, 'int_type': 'retweet'}),
 ('yamaokaro', 'rosedbarros', {'weight': 1, 'int_type': 'retweet'}),
 ('yamaokaro', 'AleGZahra38', {'weight': 1, 'int_type': 'retweet'}),
 ('yamaokaro', 'marcusPITTERnew', {'weight': 1, 'int_type': 'retweet'}),
 ('yamaokaro', 'verdeamarelou', {'weight': 1, 'int_type': 'retweet'}),
 ('a_lima74', 'renatosimoespt', {'weight': 1, 'int_type': 'retweet'})]

In [14]:
len(G_rt)

14248

## __Social Network Metrics__

### Nodes Indegree

In [15]:
# reply network nodes indegree
sorted(G_reply.in_degree, key=lambda x: x[1], reverse=True)[:10]

[('kimpaim', 675),
 ('juliette', 479),
 ('taoquei1', 413),
 ('MidiaNINJA', 393),
 ('jandira_feghali', 367),
 ('samiabomfim', 364),
 ('folha', 348),
 ('GuilhermeBoulos', 319),
 ('MarceloFreixo', 310),
 ('mjmacul_lima', 308)]

In [16]:
# retweet network nodes indegree
sorted(G_rt.in_degree, key=lambda x: x[1], reverse=True)[:10]

[('MidiaNINJA', 1290),
 ('samiabomfim', 361),
 ('GuilhermeBoulos', 331),
 ('ErikakHilton', 243),
 ('uneoficial', 214),
 ('J_LIVRES', 203),
 ('GilDoVigor', 201),
 ('RenanPeixoto_', 199),
 ('GeorgMarques', 196),
 ('jairmearrependi', 180)]

### Network Density

In [17]:
format(nx.density(G_reply), 'f')

'0.000092'

In [18]:
nx.density(G_rt)

0.0001096155590071328

### __Community Detection__

In [25]:
#rt_commun = list(greedy_modularity_communities(G_rt))
#len(rt_commun)

In [24]:
plt.figure(figsize=(15,10))

node_size = [0.01*G_rt.in_degree[v] for v in G_rt]
pos = nx.spring_layout(G_rt, k=0.15, iterations=20)
nx.draw_networkx(
    G_rt,
    pos,
    with_labels=False,
    alpha=0.8,
    node_size=node_size
)
plt.axis('off')
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …